# **Softmax Classification**
--------
Max 값을 Soft하게 붙여줘요! 와!

## **SoftMax**
분포의 종류로는, 함수의 면적이 확률 값을 나타내는 정규분포도 있고, 특정 범위에서(점) 값이 일정한 균등 분포(Uniform Distribution)도 있다.   
균등 분포를 가진 데이터를 다룰 때, SoftMax 활성화 함수를 써서 확률을 구하게 된다!
<br>

#### **균등 분포의 예)**
- **주사위 던지기(이산 균등 분포)**
    - 확률 함수가 정의돈 모든 곳에서 그 값이 일정한 분포
    - A는 0.7의 확률로 무조건(?) 가위를 낸다
- **주사위 던지기(연속 균등 분포)**
    - 특정 범위 내에서 확률이 균등하게 분포
    - 주사위를 던졌을 때 숫자 6이 나올 확률은 1/6
<br><br>

예를 들어, 친구랑 매일 천원 내기를 가위바위보 삼세판으로 한다고 치자.   
근데 내가 맨날 진다...... 분하다   
그래서 이 자식을 관찰해 보니 가위를 제일 잘 냈다.   
독이 오른 나는 얘가 가위를 냈을 때, 다음에 어떤 걸 낼지 확률을 계산해서 가장 확률이 높은 것을 내려고 한다.   
친구가 가위를 낸 다음에 각각 가위, 바위, 보를 내는 확률의 분포가 있을건데...   
모델을 학습시켜 그 확률 분포와 비슷하게 근사하고 싶다!! 이 방법으로 최대값을 소프트하게 뽑아 내는 거야!!  
&nbsp;&nbsp;&nbsp;&nbsp;👉 **SoftMax**
<br>
<br>
즉, 어떤 균등한 확률 분포가 존재할 때, 각 점에서의 확률 값을 적절한 비율로 나타낸다.  
이 비율들을 다 더하면 1이 됨!!(Sigmoid와 다른 점!!)   
SoftMax를 모델의 맨 마지막 층으로 깔면, 각각의 확률 값을 얻을 수 있기 때문에 다중 분류 문제에 사용된다.




In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

z = torch.FloatTensor([1, 2, 3]) #만약 단순하게 가장 큰 값만 뽑았다면, 해당 확률은 (0, 0, 1)이 될 것임. 하지만 소프트 맥스는 그러지 않지
hypothesis = F.softmax(z, dim = 0) #열 방향으로 확률의 합이 1이 됨
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
hypothesis.sum()

tensor(1.)

## **Cross Entropy**
2개의 확률 분포가 주어졌을 때, 그 확률 분포들이 얼마나 비슷한지 나타낼 수 있는 수치.   
근사하고 싶은 확률 분포를 향해 차이를 줄여나간다.


### **Cross Entropy Loss(Low-level)**

In [ ]:
z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1) #행을 기준으로 더해서 1
print(hypothesis) #예측값

tensor([[0.2025, 0.2176, 0.2324, 0.1142, 0.2333],
        [0.1503, 0.1465, 0.2203, 0.1962, 0.2866],
        [0.1473, 0.1262, 0.2831, 0.1827, 0.2607]], grad_fn=<SoftmaxBackward0>)


In [ ]:
#아무 수나 뽑아서 정답 만들기
y= torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 2])


In [ ]:
#정답을 one-hot vector 형태로 나타내고, 정답 인덱스에 1을, 나머지에 0을 뿌린다.
y_one_hot = torch.zeros_like(hypothesis)#(3, 5)
y_one_hot.scatter_(1, y.unsqueeze(1), 1) #(3, 5)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [ ]:
cost = (y_one_hot * - torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.4572, grad_fn=<MeanBackward0>)


### **Cross-Entropy Loss with torch.nn.functional**

In [ ]:
# -torch.log(hypothesis)와 같은 결과
torch.log(F.softmax(z, dim=1))
#아래와 같이 하나로 줄일 수 있다
F.log_softmax(z, dim=1)

tensor([[-1.5971, -1.5252, -1.4591, -2.1701, -1.4553],
        [-1.8948, -1.9205, -1.5129, -1.6285, -1.2496],
        [-1.9153, -2.0702, -1.2618, -1.6998, -1.3445]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
#(y_one_hot * -torch.log(F.softmax(z, dim=1)))와 같다
F.nll_loss(F.log_softmax(z, dim=1), y)

tensor(1.4572, grad_fn=<NllLossBackward0>)

In [ ]:
# F.softmax와 F.nll_loss를 합친 함수는 F.cross_entropy()임
F.cross_entropy(z, y)

tensor(1.4572, grad_fn=<NllLossBackward0>)

### **Training with Low-level Cross Entropy loss**

In [4]:
#label은 0, 1, 2
x_train = [[1, 2, 1, 1], # 여기서는 (8, 4)지만, (m, 4)개가 있다고 쳐보자 (m은 큰 수)
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0] #(m,)
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000

for epoch in range(1, nb_epochs+1):
    hypothesis = F.softmax(x_train.matmul(W)+b, dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * - torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.902463
Epoch  200/1000 Cost: 0.839531
Epoch  300/1000 Cost: 0.808064
Epoch  400/1000 Cost: 0.788631
Epoch  500/1000 Cost: 0.774939
Epoch  600/1000 Cost: 0.764541
Epoch  700/1000 Cost: 0.756266
Epoch  800/1000 Cost: 0.749461
Epoch  900/1000 Cost: 0.743724
Epoch 1000/1000 Cost: 0.738794


### **Training with F.cross_entropy**

In [5]:
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(1, nb_epochs+1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.714269
Epoch  200/1000 Cost: 0.638430
Epoch  300/1000 Cost: 0.591185
Epoch  400/1000 Cost: 0.554105
Epoch  500/1000 Cost: 0.522028
Epoch  600/1000 Cost: 0.492792
Epoch  700/1000 Cost: 0.465232
Epoch  800/1000 Cost: 0.438588
Epoch  900/1000 Cost: 0.412274
Epoch 1000/1000 Cost: 0.385798


### **High-level Implementation with nn.Module**

In [9]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

model = SoftmaxClassifierModel()

optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(1, nb_epochs+1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch  100/1000 Cost: 0.708336
Epoch  200/1000 Cost: 0.626136
Epoch  300/1000 Cost: 0.569848
Epoch  400/1000 Cost: 0.520366
Epoch  500/1000 Cost: 0.473517
Epoch  600/1000 Cost: 0.427770
Epoch  700/1000 Cost: 0.382397
Epoch  800/1000 Cost: 0.337085
Epoch  900/1000 Cost: 0.292189
Epoch 1000/1000 Cost: 0.251781
